In [2]:
import pandas as pd
import numpy as np

In [3]:
aisles = pd.read_csv('../data/aisles.csv')
departments = pd.read_csv('../data/departments.csv')
order_products_prior = pd.read_csv('../data/order_products__prior.csv')
order_products_train = pd.read_csv('../data/order_products__train.csv')
orders = pd.read_csv('../data/orders.csv')
products = pd.read_csv('../data/products.csv')

### 주문을 기준으로 제품 데이터 전처리

In [4]:
aisles_merge = pd.merge(products, aisles, on='aisle_id', how='left')
aisles_deapartment_merge = pd.merge(aisles_merge, departments, on='department_id', how='left')
order_products_prior_aisles_deapartment_merge = pd.merge(order_products_prior, aisles_deapartment_merge, on='product_id', how='left')
order_products_train_aisles_deapartment_merge = pd.merge(order_products_train, aisles_deapartment_merge, on='product_id', how='left')
order_products = pd.concat([order_products_prior_aisles_deapartment_merge, order_products_train_aisles_deapartment_merge]).reset_index(drop=True)
order_products

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,2,33120,1,1,Organic Egg Whites,86,16,eggs,dairy eggs
1,2,28985,2,1,Michigan Organic Kale,83,4,fresh vegetables,produce
2,2,9327,3,0,Garlic Powder,104,13,spices seasonings,pantry
3,2,45918,4,1,Coconut Butter,19,13,oils vinegars,pantry
4,2,30035,5,0,Natural Sweetener,17,13,baking ingredients,pantry
...,...,...,...,...,...,...,...,...,...
33819101,3421063,14233,3,1,Natural Artesian Water,115,7,water seltzer sparkling water,beverages
33819102,3421063,35548,4,1,Twice Baked Potatoes,13,20,prepared meals,deli
33819103,3421070,35951,1,1,Organic Unsweetened Almond Milk,91,16,soy lactosefree,dairy eggs
33819104,3421070,16953,2,1,Creamy Peanut Butter,88,13,spreads,pantry


In [5]:
order_products.columns = ['주문ID', '제품ID', '장바구니순서', '재주문', '제품이름', '1', '2', '소분류', '대분류']
주문_제품 = order_products.drop(columns=['1', '2'])
주문_제품

,주문ID,제품ID,장바구니순서,재주문,제품이름,소분류,대분류
0,2,33120,1,1,Organic Egg Whites,eggs,dairy eggs
1,2,28985,2,1,Michigan Organic Kale,fresh vegetables,produce
2,2,9327,3,0,Garlic Powder,spices seasonings,pantry
3,2,45918,4,1,Coconut Butter,oils vinegars,pantry
4,2,30035,5,0,Natural Sweetener,baking ingredients,pantry
...,...,...,...,...,...,...,...
33819101,3421063,14233,3,1,Natural Artesian Water,water seltzer sparkling water,beverages
33819102,3421063,35548,4,1,Twice Baked Potatoes,prepared meals,deli
33819103,3421070,35951,1,1,Organic Unsweetened Almond Milk,soy lactosefree,dairy eggs
33819104,3421070,16953,2,1,Creamy Peanut Butter,spreads,pantry


In [5]:
주문_제품.to_csv('../data/주문_제품.csv')

### 주문을 기준으로 대분류 빈도수 데이터 전처리

In [6]:
# 대분류 리스트 정의
categories = [
    'frozen', 'other', 'bakery', 'produce', 'alcohol', 'international',
    'beverages', 'pets', 'dry goods pasta', 'bulk', 'personal care',
    'meat seafood', 'pantry', 'breakfast', 'canned goods', 'dairy eggs',
    'household', 'babies', 'snacks', 'deli', 'missing'
]

In [7]:
# 주문ID별로 대분류를 그룹화
grouped = 주문_제품.groupby('주문ID')['대분류'].apply(lambda x: pd.Series(x).value_counts()).unstack(fill_value=0)

# 모든 대분류 열이 포함되도록 정리
grouped = grouped.reindex(columns=categories, fill_value=0)

# 결과 데이터프레임 확인
grouped

,frozen,other,bakery,produce,alcohol,international,beverages,pets,dry goods pasta,bulk,...,meat seafood,pantry,breakfast,canned goods,dairy eggs,household,babies,snacks,deli,missing
주문ID,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,4,0,0,0,0,0,0,...,0,0,0,1,3,0,0,0,0,0
2,0,0,0,3,0,0,0,0,0,0,...,0,5,0,0,1,0,0,0,0,0
3,0,0,1,3,0,0,0,0,0,0,...,1,0,0,0,3,0,0,0,0,0
4,0,0,1,0,0,0,3,0,0,0,...,0,0,4,0,0,0,0,4,0,0
5,0,0,0,7,0,1,1,0,2,0,...,1,2,0,0,3,3,0,4,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3421079,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3421080,1,0,0,2,0,0,1,0,0,0,...,0,0,0,0,5,0,0,0,0,0
3421081,0,0,1,0,0,0,1,0,0,0,...,0,1,0,1,1,0,0,1,1,0


In [8]:
주문_대분류 = grouped.reset_index()
주문_대분류

,주문ID,frozen,other,bakery,produce,alcohol,international,beverages,pets,dry goods pasta,...,meat seafood,pantry,breakfast,canned goods,dairy eggs,household,babies,snacks,deli,missing
0,1,0,0,0,4,0,0,0,0,0,...,0,0,0,1,3,0,0,0,0,0
1,2,0,0,0,3,0,0,0,0,0,...,0,5,0,0,1,0,0,0,0,0
2,3,0,0,1,3,0,0,0,0,0,...,1,0,0,0,3,0,0,0,0,0
3,4,0,0,1,0,0,0,3,0,0,...,0,0,4,0,0,0,0,4,0,0
4,5,0,0,0,7,0,1,1,0,2,...,1,2,0,0,3,3,0,4,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3346078,3421079,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3346079,3421080,1,0,0,2,0,0,1,0,0,...,0,0,0,0,5,0,0,0,0,0
3346080,3421081,0,0,1,0,0,0,1,0,0,...,0,1,0,1,1,0,0,1,1,0
3346081,3421082,0,0,0,2,0,0,0,0,0,...,1,0,1,0,2,0,0,1,0,0


In [9]:
주문_대분류.to_csv('../data/주문_대분류.csv')

### 제품 데이터 전처리

In [10]:
aisles_deapartment_merge.columns = ['제품ID', '제품이름', '1', '2', '소분류', '대분류']
제품 = aisles_deapartment_merge.drop(columns=['1', '2'])
제품

,제품ID,제품이름,소분류,대분류
0,1,Chocolate Sandwich Cookies,cookies cakes,snacks
1,2,All-Seasons Salt,spices seasonings,pantry
2,3,Robust Golden Unsweetened Oolong Tea,tea,beverages
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,frozen meals,frozen
4,5,Green Chile Anytime Sauce,marinades meat preparation,pantry
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",spirits,alcohol
49684,49685,En Croute Roast Hazelnut Cranberry,frozen vegan vegetarian,frozen
49685,49686,Artisan Baguette,bread,bakery
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,cat food care,pets


In [ ]:
제품.to_csv('../data/제품.csv')

### 주문을 기준으로 고객 데이터 전처리

In [11]:
orders.columns = ['주문ID', '고객ID', '데이터유형', '주문생성횟수', '주문생성요일', '주문생성시간', '경과일수']

# 요일 매핑 사전 생성
days_mapping = {
    0: "일요일",
    1: "월요일",
    2: "화요일",
    3: "수요일",
    4: "목요일",
    5: "금요일",
    6: "토요일"
}

# map 함수를 이용하여 '주문생성요일' 컬럼의 값을 변환
orders['주문생성요일'] = orders['주문생성요일'].map(days_mapping)

주문_고객 = orders
주문_고객

,주문ID,고객ID,데이터유형,주문생성횟수,주문생성요일,주문생성시간,경과일수
0,2539329,1,prior,1,화요일,8,NaN
1,2398795,1,prior,2,수요일,7,15.0
2,473747,1,prior,3,수요일,12,21.0
3,2254736,1,prior,4,목요일,7,29.0
4,431534,1,prior,5,목요일,15,28.0
...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,금요일,18,29.0
3421079,1854736,206209,prior,11,목요일,10,30.0
3421080,626363,206209,prior,12,월요일,12,18.0
3421081,2977660,206209,prior,13,월요일,12,7.0


In [12]:
# 주문_제품 데이터프레임에서 각 주문ID별 제품수 계산
제품수 = 주문_제품.groupby('주문ID').size().reset_index(name='제품수')

# orders 데이터프레임에 제품수 열 추가
주문_고객 = pd.merge(orders, 제품수, on='주문ID', how='left')

# 결과 출력
주문_고객

,주문ID,고객ID,데이터유형,주문생성횟수,주문생성요일,주문생성시간,경과일수,제품수
0,2539329,1,prior,1,화요일,8,NaN,5.0
1,2398795,1,prior,2,수요일,7,15.0,6.0
2,473747,1,prior,3,수요일,12,21.0,5.0
3,2254736,1,prior,4,목요일,7,29.0,5.0
4,431534,1,prior,5,목요일,15,28.0,8.0
...,...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,금요일,18,29.0,9.0
3421079,1854736,206209,prior,11,목요일,10,30.0,8.0
3421080,626363,206209,prior,12,월요일,12,18.0,20.0
3421081,2977660,206209,prior,13,월요일,12,7.0,9.0


In [13]:
# 'test' 데이터유형 제거
주문_고객 = 주문_고객[주문_고객['데이터유형'] != 'test']
주문_고객

,주문ID,고객ID,데이터유형,주문생성횟수,주문생성요일,주문생성시간,경과일수,제품수
0,2539329,1,prior,1,화요일,8,NaN,5.0
1,2398795,1,prior,2,수요일,7,15.0,6.0
2,473747,1,prior,3,수요일,12,21.0,5.0
3,2254736,1,prior,4,목요일,7,29.0,5.0
4,431534,1,prior,5,목요일,15,28.0,8.0
...,...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,금요일,18,29.0,9.0
3421079,1854736,206209,prior,11,목요일,10,30.0,8.0
3421080,626363,206209,prior,12,월요일,12,18.0,20.0
3421081,2977660,206209,prior,13,월요일,12,7.0,9.0


In [14]:
# '주문_대분류_카운팅.csv' 파일에서 '주문ID'를 기준으로 병합
merged_data = pd.merge(주문_고객, 주문_대분류, on='주문ID', how='left')
merged_data

,주문ID,고객ID,데이터유형,주문생성횟수,주문생성요일,주문생성시간,경과일수,제품수,frozen,other,...,meat seafood,pantry,breakfast,canned goods,dairy eggs,household,babies,snacks,deli,missing
0,2539329,1,prior,1,화요일,8,NaN,5.0,0,0,...,0,0,0,0,1,1,0,2,0,0
1,2398795,1,prior,2,수요일,7,15.0,6.0,0,0,...,0,0,1,0,0,0,0,3,0,0
2,473747,1,prior,3,수요일,12,21.0,5.0,0,0,...,0,1,0,0,1,0,0,2,0,0
3,2254736,1,prior,4,목요일,7,29.0,5.0,0,0,...,0,0,0,0,1,1,0,2,0,0
4,431534,1,prior,5,목요일,15,28.0,8.0,0,0,...,0,0,0,0,1,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3346078,2266710,206209,prior,10,금요일,18,29.0,9.0,1,0,...,0,1,1,0,1,1,0,1,0,0
3346079,1854736,206209,prior,11,목요일,10,30.0,8.0,0,0,...,0,0,0,0,2,1,0,1,1,0
3346080,626363,206209,prior,12,월요일,12,18.0,20.0,0,0,...,0,1,2,0,4,3,0,1,1,0
3346081,2977660,206209,prior,13,월요일,12,7.0,9.0,0,0,...,0,0,1,0,2,0,0,4,0,0


In [16]:
# 열 이름 매핑
column_mapping = {
    'frozen': '냉동식품',
    'other': '기타',
    'bakery': '제과',
    'produce': '신선식품',
    'alcohol': '주류',
    'international': '해외식품',
    'beverages': '음료',
    'pets': '애완동물',
    'dry goods pasta': '건식품,파스타',
    'bulk': '대량구매',
    'personal care': '개인위생',
    'meat seafood': '육류,해산물',
    'pantry': '식료품',
    'breakfast': '아침식사',
    'canned goods': '통조림',
    'dairy eggs': '유제품,계란',
    'household': '가정용품',
    'babies': '아기용품',
    'snacks': '간식',
    'deli': '델리',
    'missing': '누락'
}

# 열 이름 변경
merged_data.rename(columns=column_mapping, inplace=True)
merged_data

,주문ID,고객ID,데이터유형,주문생성횟수,주문생성요일,주문생성시간,경과일수,제품수,냉동식품,기타,...,"육류,해산물",식료품,아침식사,통조림,"유제품,계란",가정용품,아기용품,간식,델리,누락
0,2539329,1,prior,1,화요일,8,NaN,5.0,0,0,...,0,0,0,0,1,1,0,2,0,0
1,2398795,1,prior,2,수요일,7,15.0,6.0,0,0,...,0,0,1,0,0,0,0,3,0,0
2,473747,1,prior,3,수요일,12,21.0,5.0,0,0,...,0,1,0,0,1,0,0,2,0,0
3,2254736,1,prior,4,목요일,7,29.0,5.0,0,0,...,0,0,0,0,1,1,0,2,0,0
4,431534,1,prior,5,목요일,15,28.0,8.0,0,0,...,0,0,0,0,1,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3346078,2266710,206209,prior,10,금요일,18,29.0,9.0,1,0,...,0,1,1,0,1,1,0,1,0,0
3346079,1854736,206209,prior,11,목요일,10,30.0,8.0,0,0,...,0,0,0,0,2,1,0,1,1,0
3346080,626363,206209,prior,12,월요일,12,18.0,20.0,0,0,...,0,1,2,0,4,3,0,1,1,0
3346081,2977660,206209,prior,13,월요일,12,7.0,9.0,0,0,...,0,0,1,0,2,0,0,4,0,0


In [17]:
주문_고객 = merged_data
주문_고객

,주문ID,고객ID,데이터유형,주문생성횟수,주문생성요일,주문생성시간,경과일수,제품수,냉동식품,기타,...,"육류,해산물",식료품,아침식사,통조림,"유제품,계란",가정용품,아기용품,간식,델리,누락
0,2539329,1,prior,1,화요일,8,NaN,5.0,0,0,...,0,0,0,0,1,1,0,2,0,0
1,2398795,1,prior,2,수요일,7,15.0,6.0,0,0,...,0,0,1,0,0,0,0,3,0,0
2,473747,1,prior,3,수요일,12,21.0,5.0,0,0,...,0,1,0,0,1,0,0,2,0,0
3,2254736,1,prior,4,목요일,7,29.0,5.0,0,0,...,0,0,0,0,1,1,0,2,0,0
4,431534,1,prior,5,목요일,15,28.0,8.0,0,0,...,0,0,0,0,1,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3346078,2266710,206209,prior,10,금요일,18,29.0,9.0,1,0,...,0,1,1,0,1,1,0,1,0,0
3346079,1854736,206209,prior,11,목요일,10,30.0,8.0,0,0,...,0,0,0,0,2,1,0,1,1,0
3346080,626363,206209,prior,12,월요일,12,18.0,20.0,0,0,...,0,1,2,0,4,3,0,1,1,0
3346081,2977660,206209,prior,13,월요일,12,7.0,9.0,0,0,...,0,0,1,0,2,0,0,4,0,0


In [18]:
주문_고객.to_csv('../data/주문_고객.csv')

### 고객을 기준으로 데이터 전처리

In [30]:
# 고객ID를 기준으로 그룹화하고 주문생성요일과 주문생성시간에 대한 빈도수 계산
orders_by_customer = 주문_고객.groupby('고객ID').agg(
    월요일빈도수=('주문생성요일', lambda x: (x == '월요일').sum()),
    화요일빈도수=('주문생성요일', lambda x: (x == '화요일').sum()),
    수요일빈도수=('주문생성요일', lambda x: (x == '수요일').sum()),
    목요일빈도수=('주문생성요일', lambda x: (x == '목요일').sum()),
    금요일빈도수=('주문생성요일', lambda x: (x == '금요일').sum()),
    토요일빈도수=('주문생성요일', lambda x: (x == '토요일').sum()),
    일요일빈도수=('주문생성요일', lambda x: (x == '일요일').sum()),
)

# 0시간부터 23시간까지의 빈도수를 계산하여 추가
for hour in range(24):
    orders_by_customer[f'{hour}시간빈도수'] = 주문_고객.groupby('고객ID')['주문생성시간'].apply(lambda x: (x == hour).sum())

# 결과 출력
고객 = orders_by_customer.reset_index()
고객

,고객ID,월요일빈도수,화요일빈도수,수요일빈도수,목요일빈도수,금요일빈도수,토요일빈도수,일요일빈도수,0시간빈도수,1시간빈도수,...,14시간빈도수,15시간빈도수,16시간빈도수,17시간빈도수,18시간빈도수,19시간빈도수,20시간빈도수,21시간빈도수,22시간빈도수,23시간빈도수
0,1,3,2,2,4,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
1,2,6,5,2,1,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,3,2,1,3,0,0,0,6,0,0,...,1,2,4,2,2,1,0,0,0,0
3,4,0,0,0,2,2,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,5,1,0,2,0,0,0,2,0,0,...,0,0,1,0,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206204,206205,1,1,0,1,1,0,0,0,0,...,0,1,2,0,0,0,0,0,0,0
206205,206206,8,7,11,12,3,8,18,0,0,...,5,7,9,9,15,11,5,0,0,0
206206,206207,3,3,2,1,2,2,3,0,0,...,1,2,0,0,0,1,1,1,0,0
206207,206208,10,12,7,5,5,5,5,0,0,...,7,10,5,1,3,2,1,0,2,1


In [31]:
자정_열 = 고객[['0시간빈도수', '1시간빈도수', '2시간빈도수']]
고객['자정(0~3)'] = 자정_열.sum(axis=1)
고객.drop(columns=['0시간빈도수', '1시간빈도수', '2시간빈도수'], inplace=True)

새벽_열 = 고객[['3시간빈도수', '4시간빈도수', '5시간빈도수']]
고객['새벽(3~6)'] = 새벽_열.sum(axis=1)
고객.drop(columns=['3시간빈도수', '4시간빈도수', '5시간빈도수'], inplace=True)

이른아침_열 = 고객[['6시간빈도수', '7시간빈도수', '8시간빈도수']]
고객['이른아침(6~9)'] = 이른아침_열.sum(axis=1)
고객.drop(columns=['6시간빈도수', '7시간빈도수', '8시간빈도수'], inplace=True)

늦은오전_열 = 고객[['9시간빈도수', '10시간빈도수', '11시간빈도수']]
고객['늦은오전(9~12)'] = 늦은오전_열.sum(axis=1)
고객.drop(columns=['9시간빈도수', '10시간빈도수', '11시간빈도수'], inplace=True)

점심시간_열 = 고객[['12시간빈도수', '13시간빈도수']]
고객['점심시간(12~14)'] = 점심시간_열.sum(axis=1)
고객.drop(columns=['12시간빈도수', '13시간빈도수'], inplace=True)

오후초기_열 = 고객[['14시간빈도수', '15시간빈도수', '16시간빈도수']]
고객['오후초기(14~17)'] = 오후초기_열.sum(axis=1)
고객.drop(columns=['14시간빈도수', '15시간빈도수', '16시간빈도수'], inplace=True)

늦은오후_열 = 고객[['17시간빈도수', '18시간빈도수']]
고객['늦은오후(17~19)'] = 늦은오후_열.sum(axis=1)
고객.drop(columns=['17시간빈도수', '18시간빈도수'], inplace=True)

저녁_열 = 고객[['19시간빈도수', '20시간빈도수', '21시간빈도수']]
고객['저녁(19~22)'] = 저녁_열.sum(axis=1)
고객.drop(columns=['19시간빈도수', '20시간빈도수', '21시간빈도수'], inplace=True)

늦은밤_열 = 고객[['22시간빈도수', '23시간빈도수']]
고객['늦은밤(22~24)'] = 늦은밤_열.sum(axis=1)
고객.drop(columns=['22시간빈도수', '23시간빈도수'], inplace=True)


In [32]:
# 각 고객ID에 대해 주문생성횟수의 최대값을 계산
max_order_count = orders.groupby('고객ID')['주문생성횟수'].max().reset_index()
max_order_count.columns = ['고객ID', '총주문횟수']

# 고객 데이터프레임에 총주문횟수 열 추가
고객 = pd.merge(고객, max_order_count, on='고객ID', how='left')
고객

,고객ID,월요일빈도수,화요일빈도수,수요일빈도수,목요일빈도수,금요일빈도수,토요일빈도수,일요일빈도수,자정(0~3),새벽(3~6),이른아침(6~9),늦은오전(9~12),점심시간(12~14),오후초기(14~17),늦은오후(17~19),저녁(19~22),늦은밤(22~24),총주문횟수
0,1,3,2,2,4,0,0,0,0,0,6,1,1,3,0,0,0,11
1,2,6,5,2,1,1,0,0,0,0,0,13,1,1,0,0,0,15
2,3,2,1,3,0,0,0,6,0,0,0,0,0,7,4,1,0,13
3,4,0,0,0,2,2,1,0,0,0,0,2,2,1,0,0,0,6
4,5,1,0,2,0,0,0,2,0,0,0,1,1,1,2,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206204,206205,1,1,0,1,1,0,0,0,0,0,0,1,3,0,0,0,4
206205,206206,8,7,11,12,3,8,18,0,0,0,1,5,21,24,16,0,68
206206,206207,3,3,2,1,2,2,3,0,1,2,4,3,3,0,3,0,17
206207,206208,10,12,7,5,5,5,5,0,1,1,10,5,22,4,3,3,50


In [33]:
# 각 고객ID에 대해 주문생성횟수의 최대값에 해당하는 경과일수 계산
last_order_days = orders.loc[orders.groupby('고객ID')['주문생성횟수'].idxmax(), ['고객ID', '경과일수']]
last_order_days.columns = ['고객ID', '마지막주문(경과일수)']

# 고객 데이터프레임에 마지막주문(경과일수) 열 추가
고객 = pd.merge(고객, last_order_days, on='고객ID', how='left')

# 결과 출력
고객

,고객ID,월요일빈도수,화요일빈도수,수요일빈도수,목요일빈도수,금요일빈도수,토요일빈도수,일요일빈도수,자정(0~3),새벽(3~6),이른아침(6~9),늦은오전(9~12),점심시간(12~14),오후초기(14~17),늦은오후(17~19),저녁(19~22),늦은밤(22~24),총주문횟수,마지막주문(경과일수)
0,1,3,2,2,4,0,0,0,0,0,6,1,1,3,0,0,0,11,14.0
1,2,6,5,2,1,1,0,0,0,0,0,13,1,1,0,0,0,15,30.0
2,3,2,1,3,0,0,0,6,0,0,0,0,0,7,4,1,0,13,11.0
3,4,0,0,0,2,2,1,0,0,0,0,2,2,1,0,0,0,6,30.0
4,5,1,0,2,0,0,0,2,0,0,0,1,1,1,2,0,0,5,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206204,206205,1,1,0,1,1,0,0,0,0,0,0,1,3,0,0,0,4,10.0
206205,206206,8,7,11,12,3,8,18,0,0,0,1,5,21,24,16,0,68,0.0
206206,206207,3,3,2,1,2,2,3,0,1,2,4,3,3,0,3,0,17,14.0
206207,206208,10,12,7,5,5,5,5,0,1,1,10,5,22,4,3,3,50,4.0


In [34]:
# 각 고객ID에 대해 경과일수의 합계 계산 (NaN 값은 무시)
used_days = orders.groupby('고객ID')['경과일수'].sum(min_count=1).reset_index()
used_days.columns = ['고객ID', '사용날짜']

# 고객 데이터프레임에 사용날짜 열 추가
고객 = pd.merge(고객, used_days, on='고객ID', how='left')

# 결과 출력
고객

,고객ID,월요일빈도수,화요일빈도수,수요일빈도수,목요일빈도수,금요일빈도수,토요일빈도수,일요일빈도수,자정(0~3),새벽(3~6),이른아침(6~9),늦은오전(9~12),점심시간(12~14),오후초기(14~17),늦은오후(17~19),저녁(19~22),늦은밤(22~24),총주문횟수,마지막주문(경과일수),사용날짜
0,1,3,2,2,4,0,0,0,0,0,6,1,1,3,0,0,0,11,14.0,190.0
1,2,6,5,2,1,1,0,0,0,0,0,13,1,1,0,0,0,15,30.0,228.0
2,3,2,1,3,0,0,0,6,0,0,0,0,0,7,4,1,0,13,11.0,144.0
3,4,0,0,0,2,2,1,0,0,0,0,2,2,1,0,0,0,6,30.0,85.0
4,5,1,0,2,0,0,0,2,0,0,0,1,1,1,2,0,0,5,6.0,46.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206204,206205,1,1,0,1,1,0,0,0,0,0,0,1,3,0,0,0,4,10.0,50.0
206205,206206,8,7,11,12,3,8,18,0,0,0,1,5,21,24,16,0,68,0.0,249.0
206206,206207,3,3,2,1,2,2,3,0,1,2,4,3,3,0,3,0,17,14.0,229.0
206207,206208,10,12,7,5,5,5,5,0,1,1,10,5,22,4,3,3,50,4.0,361.0


In [35]:
# orders 데이터프레임에서 고객별 경과일수의 평균 계산
평균경과일수 = orders.groupby('고객ID')['경과일수'].mean().reset_index()
평균경과일수.columns = ['고객ID', '평균경과일수']

# 고객 데이터프레임에 평균경과일수 열 추가
고객 = pd.merge(고객, 평균경과일수, on='고객ID', how='left')

# 결과 출력
고객

,고객ID,월요일빈도수,화요일빈도수,수요일빈도수,목요일빈도수,금요일빈도수,토요일빈도수,일요일빈도수,자정(0~3),새벽(3~6),...,늦은오전(9~12),점심시간(12~14),오후초기(14~17),늦은오후(17~19),저녁(19~22),늦은밤(22~24),총주문횟수,마지막주문(경과일수),사용날짜,평균경과일수
0,1,3,2,2,4,0,0,0,0,0,...,1,1,3,0,0,0,11,14.0,190.0,19.000000
1,2,6,5,2,1,1,0,0,0,0,...,13,1,1,0,0,0,15,30.0,228.0,16.285714
2,3,2,1,3,0,0,0,6,0,0,...,0,0,7,4,1,0,13,11.0,144.0,12.000000
3,4,0,0,0,2,2,1,0,0,0,...,2,2,1,0,0,0,6,30.0,85.0,17.000000
4,5,1,0,2,0,0,0,2,0,0,...,1,1,1,2,0,0,5,6.0,46.0,11.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206204,206205,1,1,0,1,1,0,0,0,0,...,0,1,3,0,0,0,4,10.0,50.0,16.666667
206205,206206,8,7,11,12,3,8,18,0,0,...,1,5,21,24,16,0,68,0.0,249.0,3.716418
206206,206207,3,3,2,1,2,2,3,0,1,...,4,3,3,0,3,0,17,14.0,229.0,14.312500
206207,206208,10,12,7,5,5,5,5,0,1,...,10,5,22,4,3,3,50,4.0,361.0,7.367347


In [36]:
# 고객별 총 제품수 계산
고객별_총_제품수 = 주문_고객.groupby('고객ID')['제품수'].sum().reset_index()

# 고객 데이터프레임에 고객별 총 제품수 추가
고객 = pd.merge(고객, 고객별_총_제품수, on='고객ID', suffixes=('', '_총'))

고객

,고객ID,월요일빈도수,화요일빈도수,수요일빈도수,목요일빈도수,금요일빈도수,토요일빈도수,일요일빈도수,자정(0~3),새벽(3~6),...,점심시간(12~14),오후초기(14~17),늦은오후(17~19),저녁(19~22),늦은밤(22~24),총주문횟수,마지막주문(경과일수),사용날짜,평균경과일수,제품수
0,1,3,2,2,4,0,0,0,0,0,...,1,3,0,0,0,11,14.0,190.0,19.000000,70.0
1,2,6,5,2,1,1,0,0,0,0,...,1,1,0,0,0,15,30.0,228.0,16.285714,226.0
2,3,2,1,3,0,0,0,6,0,0,...,0,7,4,1,0,13,11.0,144.0,12.000000,88.0
3,4,0,0,0,2,2,1,0,0,0,...,2,1,0,0,0,6,30.0,85.0,17.000000,18.0
4,5,1,0,2,0,0,0,2,0,0,...,1,1,2,0,0,5,6.0,46.0,11.500000,46.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206204,206205,1,1,0,1,1,0,0,0,0,...,1,3,0,0,0,4,10.0,50.0,16.666667,51.0
206205,206206,8,7,11,12,3,8,18,0,0,...,5,21,24,16,0,68,0.0,249.0,3.716418,285.0
206206,206207,3,3,2,1,2,2,3,0,1,...,3,3,0,3,0,17,14.0,229.0,14.312500,223.0
206207,206208,10,12,7,5,5,5,5,0,1,...,5,22,4,3,3,50,4.0,361.0,7.367347,677.0


In [37]:
# 21개의 카테고리 열 이름
category_columns = [
    '냉동식품', '기타', '제과', '신선식품', '주류', '해외식품', '음료', '애완동물', 
    '건식품,파스타', '대량구매', '개인위생', '육류,해산물', '식료품', '아침식사', 
    '통조림', '유제품,계란', '가정용품', '아기용품', '간식', '델리', '누락'
]

# 고객ID별로 21개 열을 합산하여 새로운 데이터프레임 생성
고객_대분류 = 주문_고객.groupby('고객ID')[category_columns].sum().reset_index()

# 결과 확인
고객_대분류

,고객ID,냉동식품,기타,제과,신선식품,주류,해외식품,음료,애완동물,"건식품,파스타",...,"육류,해산물",식료품,아침식사,통조림,"유제품,계란",가정용품,아기용품,간식,델리,누락
0,1,0,0,0,5,0,0,15,0,0,...,0,1,4,0,17,3,0,25,0,0
1,2,28,0,2,43,0,3,9,0,0,...,1,11,3,4,49,0,0,47,24,0
2,3,6,0,0,38,0,0,3,0,4,...,0,4,0,0,21,1,0,9,2,0
3,4,3,0,2,2,2,0,3,0,0,...,0,0,0,1,0,2,0,1,2,0
4,5,2,0,0,23,0,4,0,0,1,...,0,3,0,1,9,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206204,206205,3,0,2,14,0,1,1,0,0,...,1,1,0,0,20,0,1,0,5,2
206205,206206,76,1,2,34,0,4,33,0,1,...,2,14,1,9,39,12,0,42,5,0
206206,206207,21,0,3,50,0,1,20,0,8,...,6,15,4,11,52,0,1,23,8,0
206207,206208,26,0,62,197,0,3,20,0,17,...,22,38,17,11,161,6,3,59,28,2


In [38]:
# '고객_대분류_카운팅.csv' 파일에서 '고객ID'를 기준으로 병합
merged_data = pd.merge(고객, 고객_대분류, on='고객ID', how='left')
merged_data

,고객ID,월요일빈도수,화요일빈도수,수요일빈도수,목요일빈도수,금요일빈도수,토요일빈도수,일요일빈도수,자정(0~3),새벽(3~6),...,"육류,해산물",식료품,아침식사,통조림,"유제품,계란",가정용품,아기용품,간식,델리,누락
0,1,3,2,2,4,0,0,0,0,0,...,0,1,4,0,17,3,0,25,0,0
1,2,6,5,2,1,1,0,0,0,0,...,1,11,3,4,49,0,0,47,24,0
2,3,2,1,3,0,0,0,6,0,0,...,0,4,0,0,21,1,0,9,2,0
3,4,0,0,0,2,2,1,0,0,0,...,0,0,0,1,0,2,0,1,2,0
4,5,1,0,2,0,0,0,2,0,0,...,0,3,0,1,9,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206204,206205,1,1,0,1,1,0,0,0,0,...,1,1,0,0,20,0,1,0,5,2
206205,206206,8,7,11,12,3,8,18,0,0,...,2,14,1,9,39,12,0,42,5,0
206206,206207,3,3,2,1,2,2,3,0,1,...,6,15,4,11,52,0,1,23,8,0
206207,206208,10,12,7,5,5,5,5,0,1,...,22,38,17,11,161,6,3,59,28,2


In [39]:
고객 = merged_data
고객

,고객ID,월요일빈도수,화요일빈도수,수요일빈도수,목요일빈도수,금요일빈도수,토요일빈도수,일요일빈도수,자정(0~3),새벽(3~6),...,"육류,해산물",식료품,아침식사,통조림,"유제품,계란",가정용품,아기용품,간식,델리,누락
0,1,3,2,2,4,0,0,0,0,0,...,0,1,4,0,17,3,0,25,0,0
1,2,6,5,2,1,1,0,0,0,0,...,1,11,3,4,49,0,0,47,24,0
2,3,2,1,3,0,0,0,6,0,0,...,0,4,0,0,21,1,0,9,2,0
3,4,0,0,0,2,2,1,0,0,0,...,0,0,0,1,0,2,0,1,2,0
4,5,1,0,2,0,0,0,2,0,0,...,0,3,0,1,9,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206204,206205,1,1,0,1,1,0,0,0,0,...,1,1,0,0,20,0,1,0,5,2
206205,206206,8,7,11,12,3,8,18,0,0,...,2,14,1,9,39,12,0,42,5,0
206206,206207,3,3,2,1,2,2,3,0,1,...,6,15,4,11,52,0,1,23,8,0
206207,206208,10,12,7,5,5,5,5,0,1,...,22,38,17,11,161,6,3,59,28,2


In [40]:
고객.to_csv('../data/고객.csv')